In [ ]:
import sqlite3
import csv

This file is currently just for testing purposes---plays with a placeholder sqlite3 database that has columns "test_1" and "test_2"

In [ ]:
def select_and_print(cursor, command_string):
    """
    Selects entries from database and prints it out to the python interpreter. 
    
    Parameters: 
    cursor         : the sqlite3 cursor created from .cursor() on a connected sqlite 3 database
    command_string : the entire select SQL command (e.g. "select * from testing123;")
    """
    cursor.execute(command_string)

    rows = cursor.fetchall()
    for row in rows:
        print(row.keys())

        string = ""
        for key in row.keys():
            string += str(row[key])
            string += " | "
        # print(str(row['test_1']) + " | " + str(row['test_2']))

        print(string)
        
    # print(rows.keys())
    
    # print(rows[0]['test_1'])
    # print(rows[0]['test_2'])
    print("---------------")